<font color='orange' size=8><B>**Lab Report 2 : Robot Modelling & Kinematics**</B></font>

<font size=5><B>Objective :</B></font>

การทดลองนี้ออกแบบมาเพื่อการเรียนรู้และประยุกต์ใช้ความรู้ในด้าน robot modelling และ kinematics สำหรับการประยุกต์เข้ากับ ROS2

------------------------------------------------------------------------------------------------------------------------------

<font size=5><B>Summary on Position Kinematics:</B></font>

$$
\newcommand{\pvec}[4]{\mathbf{#1}_{\mathrm{#2},\mathrm{#3}}^{\mathrm{#4}}}
$$

$$
\def\rvec#1#2#3{\mathbf{#1}_{\mathrm{#2}}^{\mathrm{#3}}}
\def\rotm#1#2#3{\mathbf{#1}_{\mathrm{#2}}^{\mathrm{#3}}}
\def\uvec#1#2#3{\hat{\mathbf{#1}}_{\mathrm{#2}}^{\mathrm{#3}}}
$$



ในจลนศาสตร์ตำแหน่ง **TASKSPACE** คือตำแหน่งทั้งหมดที่เป็นไปได้ของตำแหน่งปลายมือที่อ้างอิงกับเฟรมโลก (global coordinate frame) ในระบบพิกัดฉาก และ **CONFIGURATION SPACE** คือตำแหน่งของข้อต่อ (joint configuration) ทั้งหมดที่เป็นไปได้ของหุ่นยนต์ ดังนั้นถ้าหากนักศึกษาทราบค่า joint configuration ของหุ่นยนต์ในแต่ละข้อต่อจะสามารถแปลงให้เป็นตำแหน่งปลายมือของหุ่นยนต์ด้วยสมการ **FORWARD KINEMATICS** ดังสมการต่อไปนี้

$$
\pvec{p}{G}{e}{G} = \mathbf{fk}(\mathbf{q})
$$

   เมื่อ $\mathbf{q}$ คือตำแหน่งข้อต่อของหุ่นยนต์และ $\pvec{p}{G}{e}{G}$ คือตำแหน่งปลายมือของหุ่นยนต์


หรืออีกอย่างหนึ่ง เมื่อเราทราบตำแหน่งข้อต่อของหุ่นยนต์จะสามารถใช้ forward kinematics ในการแก้ไขปัญหาเพื่อหาค่าตำแหน่งปลายมือของหุ่นยนต์

ในทางตรงกันข้าม การแปลงตำแหน่งปลายมือของหุ่นยนต์ให้เป็นตำแหน่งข้อต่อของหุ่นยนต์สามารถทำได้โดยการใช้สมการ **INVERSE KINEMATICS** โดยใช้สมการดังต่อไปนี้

$$
\mathbf{q} = \mathbf{ik}(\pvec{p}{G}{e}{G})
$$

<center><img src="pic/hw2-1.png" width="400"/></center>

แม้ว่าการแปลงจากตำแหน่งข้อต่อไปยังตำแหน่งปลายมือของหุ่นยนต์จะมีคำตอบที่เป็นไปได้อยู่คำตอบเดียว แต่ในการแปลงตำแหน่งปลายมือไปยังตำแหน่งข้อต่อของหุ่นยนต์นั้นจะมีหลายคำตอบด้วยกัน ดังนั้นจะมีตัวแปรดำอ่มขึ้นมาตัวหนึ่งสำหรับอธิบายท่าทางของหุ่นยนต์ที่มีหลายคำตอบ "**ARM CONFIGURATION**" ดังนี้ $\mathbf{\gamma}\in{\{-1,1\}}^m$ ใช้สำหรับการอธิบายท่าทางของคำตอบที่ได้จากการใช้สมการ inverse kinematics


$$
\mathbf{q} = \mathbf{ik}(\pvec{p}{G}{e}{G},\mathbf{\gamma})
$$

<font size=5><B>Summary on Robot Visualization Pipeline in ROS2:</B></font>

ใน ROS2 นั้นมีโปรแกรมตัวหนึ่งที่ชื่อว่า *rviz2* ที่เป็นเครื่องมือสำหรับการแสดงผล *VISUALIZE* ข้อมูลที่ส่งผ่านทาง topic ใน ROS2 network โดยการระบุชื่อ topic และประเภทของข้อมูล หนึ่งในประเภท topic ที่สามารถแสดงผลรูปร่างลักษณะของหุ่นยนต์คือ "robot_description" ซึ่งถือว่าเป็น topic ที่เป็นจุดเด่นของการใช้งาน *rviz2* ถึงแม้ว่า topic ของ "robot_description" จะเป็นประเภท std_msgs/String และต้องอยู่ในรูปแบบของ XML เท่านั้น มิฉะนั้น rviz2 จะไม่สามารถแสดงผลได้ ในทางทฤษฎีเราสามารถเขียนตัวแปร std_msgs/String ให้อยู่ในรูปแบบของ XML และส่งค่าโดยการ publish ไปยัง *rviz2* ได้โดยตรง แต่ ROS2 นั้นมีวิธีการทำ robot_description ที่ดีกว่านี้

ในการแสดงผลหุ่นนยนต์จะต้องอาศัย topic อื่นประกอบด้วยคือ "Transformation Tree (TF_Tree)" ซึ่งถูกนำมาใช้เป็นแกนหลักของหุ่นยนต์เพื่อใช้หา forward kinematics สำหรับหาตำแหน่งเฟรมของหุ่นยนต์ในแต่ละเฟรมในเทอมของตำแหน่ง ตำแหน่งเชิงมุมและเวลา timestamp ที่ถูกส่งออกไปในแต่ละครั้ง เพื่อให้เกิดประสิทธิภาพสูงสุด ROS2 ได้พัฒนาการส่งข้อมูลของ TF_Tree ออกเป็น 2 ประเภทด้วยกันคือ */tf_static* และ */tf* โดยที่ transformation(tf) ที่ไม่เปลี่ยนแปลงตามเวลาจะส่งออกผ่าน topic */tf_static* ถ้า tf ที่เปลี่ยนแปลงระหว่างเฟรมตามเวลาจะถูกส่งออกผ่าน topic */tf* *rviz2* จะใช้ topics ทั้งสองรวมกันเพื่อสร้าง TF_TREE ที่สมบูรณ์ ในบางครั้งผู้ใช้งานสามารถพัฒนาโหนดเพื่อส่ง (broadcast) tf ไปยัง TF_TREE แบบ manual ในทางเทคนิคเราสามารถเขียนโปรแกรมให้ส่งค่า TF_TREE สำหรับแสดงผลของหุ่นยนต์ โดยที่ ROS2 มีเครื่องมือที่สะดวกสำหรับการ broadcast TF_TREE แบบอัตโนมัติตามการเขียนความสัมพันธ์จาก "robot_description"

ในการเขียนไฟล์ XML สำหรับการอธิบายลักษณะของหุ่นยนต์และการส่ง tf แบบ manual นั้นเป็นไปได้ยาก แต่โชคยังดีที่ใน ROS2 นั้นมี node หนึ่งชื่อว่า *robot_state_publisher* ที่สามารถรวมค่าของข้อต่อหุ่นยนต์ "/joint_states" และไฟล์ URDF จากนั้นส่งค่าของ TF_TREE และ "robot_description" ไปยัง *rviz2*


<center><img src="pic/hw2-2.png" width="600"/></center>

------------------------------------------------------------------------------------------------------------------------------

<font color='orange' size=6><B>ภาพรวมของหุ่นยนต์(Cocoa)ที่ได้ออกแบบมา</B></font>

<center><img src="img/DH.jpg"/></center>


โดยเราออกแบบให้หุ่นยนต์ของพวกเราประกอบด้วย ข้อต่อแบบ Revolute ทั้งหมด 4 ข้อต่อและข้อต่อแบบ Fixed 1 ข้อต่อโดยจะสามารถจัดเรียงได้ดังนี้
        <ul>
        <li>/base_link</li>
        <li>/"joint_rev_b_0" type="revolute"</li>
        <li>/link_0</li>
        <li>/"joint_rev_0_1" type="revolute"</li>
        <li>/link_1</li>
        <li>/"joint_rev_1_2" type="revolute"</li>
        <li>/link_2</li>
        <li>/"joint_rev_2_3" type="revolute"</li>
        <li>/link_3</li>
        <li>/"joint_eff" type="fixed"</li>
        <li>/end_effector</li>     
        </ul>  


------------------------------------------------------------------------------------------------------------------------------

<font color='orange' size=6><B>การแสดงผลของแบบจำลอง</B></font>

1.การเขียนไฟล์ Display<br>

1.1 ทำการดึงค่า Setting ของโปรแกรม Rviz ที่ได้ save เอาไว้ในไฟล์ cocoa.rviz

<img src="img/display1.png"/>

1.2 ทำการดึง Path ของไฟล์ Cocoa และใช้ Xacro ในการเก็บของมูล

<img src="img/display2.png"/>

1.3 ทำการสร้าง Node เพื่อ publish ค่าของข้อต่อหุ่นยนต์ "/joint_states" และไฟล์ URDF(Xacro) จากนั้นส่งค่าของ TF_TREE และ "robot_description" ไปยัง *rviz2*

<img src="img/display3.png"/>

1.4 ทำการเพิ่ม Launch Description Action ที่สร้างมาทั้งหมด

<img src="img/display4.png"/></br>


------------------------------------------------------------------------------------------------------------------------------

<font color='orange' size=6><B>พัฒนาโหนด kinematics server</B></font>


1.การสร้าง Custom interface ของ Cocoa

1.1 ทำการสร้างไฟล์ Cocoa.srv โดยจะกำหนด input และ output ของ Cocoa โดยจะมี 3 ตัวแปร คือ ตัวแปร x,y,z และ ตัวแปร ของข้อต่อ 4 ตัวแปร โดยจะมีค่าเป็น float64 และจะมีค่า output ที่เป็น 4 ตัวแปร โดยจะมีค่าเป็น float64 ดังนี้

------------------------------------------------------------------------------------------------------------------------------

<font color='orange' size=6><B>ตาราง DH PARAMETERS ของ Cocoa ณ Home configuration</B></font>

<center><img src="img/DH.jpg"/></center>

------------------------------------------------------------------------------------------------------------------------------

<font color='orange' size=6><B>การคำนวณ Forward Kinematic ของ Cocoa</B></font>

In [ ]:
import sympy as sp

x = sp.Symbol("x", positive = True)
y = sp.Symbol("y", positive = True)
z = sp.Symbol("z", positive = True)
l1 = sp.Symbol("l1", positive = True)
l2 = sp.Symbol("l2", positive = True)
l3 = sp.Symbol("l3", positive = True)
l4 = sp.Symbol("l4", positive = True)
q1 = sp.Symbol("q1", positive = True)
q2 = sp.Symbol("q2", positive = True)
q3 = sp.Symbol("q3", positive = True)
q4 = sp.Symbol("q4", positive = True)

phi = sp.Symbol("phi", positive = True)
pie = sp.Symbol("pi", positive = True)

In [ ]:
def translateX(x):
    return sp.Matrix([[1, 0, 0, x], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

def translateZ(z):
    return sp.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, z], [0, 0, 0, 1]])

def rotateX(theta):
    return sp.Matrix([[1, 0, 0, 0], [0, sp.cos(theta), -sp.sin(theta), 0], [0, sp.sin(theta), sp.cos(theta), 0], [0, 0, 0, 1]])

def rotateZ(theta):
    return sp.Matrix([[sp.cos(theta), -sp.sin(theta), 0, 0], [sp.sin(theta), sp.cos(theta), 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])

def rotateY(theta):
    return sp.Matrix([[sp.cos(theta), 0, sp.sin(theta), 0], [0, 1, 0, 0], [-sp.sin(theta), 0, sp.cos(theta), 0], [0, 0, 0, 1]])

In [ ]:
DH = sp.Matrix([[0, 0, l1, 0, 0],
                [0, sp.pi/2, 0, sp.pi/2, 0],
                [l2, 0, 0, -sp.pi/2, 0],
                [l3, 0, 0, 0,0]])
DH

In [ ]:
H  = sp.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
q = [q1,q2,q3,q4]
for i in range(DH.shape[0]):
    if DH[i, 4] == 0:# Revolute joint
        Hj = rotateZ(q[i])
    elif DH[i, 4] == 1: # Prismatic joint
        Hj = translateZ(q[i])
    else: # Fixed joint
        Hj = sp.Matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    H = H * translateX(DH[i, 0]) * rotateX(DH[i, 1]) * translateZ(DH[i, 2]) * rotateZ(DH[i, 3]) * Hj
H = sp.trigsimp(H)
H


In [ ]:
H4e = sp.Matrix([[1, 0, 0, l4],
                 [0, 0, 1, 0],
                 [0, -1, 0, 0],
                 [0, 0, 0, 1]])
He = H * H4e
He = sp.trigsimp(He)

In [ ]:
P = sp.trigsimp(He.col(3).row([0,1,2]))
eqP = sp.Eq(sp.Matrix([x, y, z]),P)
eqP